
## Quantum State Preparation from Arbitrary Complex Amplitudes

This notebook implements **Task 2: Complex Amplitudes** from the QOSF Cohort 11 screening tasks.

### Objective
To prepare a two-qubit quantum state $|\psi\rangle = a_0|00\rangle + a_1|01\rangle + a_2|10\rangle + a_3|11\rangle$
from an arbitrary list of four complex amplitudes $[a_0, a_1, a_2, a_3]$, without using any high-level state initialization
functions from quantum libraries such as Qiskit or PennyLane.

We use only NumPy to represent gates and states via matrices and vectors.



### Approach

1. **Normalization**
   - Ensure the input amplitudes form a normalized state: $|a_0|^2 + |a_1|^2 + |a_2|^2 + |a_3|^2 = 1$.
   - If not, divide all amplitudes by their Euclidean norm.

2. **Block decomposition**
   - Group amplitudes according to the first qubit’s value:
     - $\psi_0 = [a_0, a_1]$ (when the first qubit is |0⟩)
     - $\psi_1 = [a_2, a_3]$ (when the first qubit is |1⟩)

3. **Prepare first qubit**
   - Define $\alpha_0 = ||\psi_0||$, $\alpha_1 = ||\psi_1||$.
   - Rotate the first qubit from $|0\rangle$ to $\alpha_0|0\rangle + \alpha_1|1\rangle$ using a single-qubit $RY(\theta)$ gate, where
     $\theta = 2\arctan(\alpha_1 / \alpha_0)$.

4. **Prepare second qubit conditionally**
   - Build two single-qubit unitaries $U_0$ and $U_1$ such that:
     - $U_0|0\rangle = \psi_0 / \alpha_0$
     - $U_1|0\rangle = \psi_1 / \alpha_1$
   - Create a block-diagonal controlled unitary $CU = \text{diag}(U_0, U_1)$.

5. **Combine operations**
   - Apply the total unitary:
     $U = CU (RY(\theta) \otimes I)$
     on the initial state |00⟩.

6. **Verification**
   - Verify that $U|00\rangle$ matches the normalized target state.


In [3]:
# Import the NumPy library for numerical and matrix operations
import numpy as np
from numpy import linalg as LA # 'LA' is just a shorter alias for linear algebra functions like norm

# -----------------------------
# 1. Normalization function
# -----------------------------
def normalize(amps):
    """
    Normalize a list of complex amplitudes so that the total probability equals 1.
    
    Args:
        amps (list or array): list of complex amplitudes [a0, a1, a2, a3]
    Returns:
        np.ndarray: normalized complex vector such that sum(|ai|^2) = 1
    """
    vec = np.array(amps, dtype=np.complex128).reshape(-1) # Convert to a 1D NumPy array of complex numbers
    norm = LA.norm(vec) # Compute the vector norm (Euclidean norm = sqrt(sum(|ai|^2)))

    if norm == 0:
         # If all amplitudes are zero, normalization is impossible (invalid input)
        raise ValueError("Input amplitudes are all zero; cannot normalize.")
    
    return vec / norm # Divide by the norm → ensures the total probability = 1

# -----------------------------
# 2. Single-qubit RY rotation
# ----------------------------
def single_qubit_RY(theta):
    """
    Construct the single-qubit RY rotation gate.
    
    Formula:
    RY(theta) = [[cos(theta/2), -sin(theta/2)],
                 [sin(theta/2),  cos(theta/2)]]
    
    Args:
        theta (float): rotation angle in radians
    Returns:
        np.ndarray: 2x2 complex matrix representing RY(theta)
    """
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2),  np.cos(theta/2)]], dtype=np.complex128)

# -----------------------------
# 3. Tensor product (Kronecker product)
# -----------------------------
def kron2(A, B):
    """
    Compute the tensor (Kronecker) product of two matrices A and B.
    
    This is how we represent multi-qubit operators in matrix form.
    Example: kron2(RY, I) applies RY to the first qubit and I (identity) to the second.
    """
    return np.kron(A, B)

# -----------------------------
# 4. Construct a 2x2 unitary that maps |0> -> v
# -----------------------------
def make_unitary_from_target(v):
    """
    Build a single-qubit unitary U such that U|0> = v.
    
    Args:
        v (array): 2-element complex vector (normalized)
    Returns:
        np.ndarray: 2x2 unitary matrix with first column = v
    """
    v = np.array(v, dtype=np.complex128).reshape(2) # Ensure v is a 2-element column vector
    v = v / LA.norm(v) # Normalize v (safety)

    # Find a second vector 'w' orthogonal to v to complete the unitary basis
    # If v = [v0, v1], then w = [-conj(v1), conj(v0)] is orthogonal to v
    w = np.array([-np.conj(v[1]), np.conj(v[0])], dtype=np.complex128)
    w = w / LA.norm(w)

    # Stack v and w as columns → gives a 2x2 unitary matrix U = [v | w]
    U = np.column_stack([v, w])
    return U

# -----------------------------
# 5. Main function: Prepare two-qubit state
# -----------------------------
def prepare_two_qubit_state(amplitudes):
    """
    Prepare a two-qubit state |ψ> = a0|00> + a1|01> + a2|10> + a3|11>
    using only basic linear algebra (no high-level quantum functions).
    """

    # Step 1: Normalize input amplitudes
    a = normalize(amplitudes)

    # Step 2: Split into sub-vectors for first qubit = 0 and first qubit = 1
    # psi0 → amplitudes where first qubit = 0 (|00>, |01>)
    # psi1 → amplitudes where first qubit = 1 (|10>, |11>)
    psi0, psi1 = a[0:2], a[2:4]

    # Step 3: Compute norms (magnitudes) of psi0 and psi1
    # These determine how much probability mass is in each "branch"
    alpha0, alpha1 = LA.norm(psi0), LA.norm(psi1)

    # Step 4: Handle edge cases — if both are zero, the input was invalid
    if alpha0 < 1e-12 and alpha1 < 1e-12:
        raise ValueError("All amplitudes zero after normalization.")
    
    # Step 5: Compute RY rotation angle for the first qubit
    # We want RY(theta)|0> = alpha0|0> + alpha1|1>
    # So, theta = 2 * atan(alpha1 / alpha0)
    theta = 0.0 if alpha1 < 1e-12 else 2 * np.arctan2(alpha1, alpha0)

    # Build the single-qubit RY gate for the first qubit
    RY0 = single_qubit_RY(theta)

    # Step 6: Build conditional unitaries for the second qubit
    # U0 maps |0> → normalized psi0 (when first qubit = 0)
    # U1 maps |0> → normalized psi1 (when first qubit = 1)
    U0 = np.eye(2) if alpha0 < 1e-12 else make_unitary_from_target(psi0 / alpha0)
    U1 = np.eye(2) if alpha1 < 1e-12 else make_unitary_from_target(psi1 / alpha1)

    # Step 7: Construct a 4x4 block-diagonal controlled unitary
    # CU = diag(U0, U1) — applies U0 if first qubit=0, U1 if first qubit=1
    CU = np.block([[U0, np.zeros((2,2))],
                   [np.zeros((2,2)), U1]])
    
    # Step 8: Combine everything
    # First apply RY on qubit 0, then the controlled unitary on both qubits
    U_full = CU @ kron2(RY0, np.eye(2))

    # Step 9: Apply the full unitary to |00> = [1, 0, 0, 0]^T
    output = U_full @ np.array([1,0,0,0], dtype=np.complex128)

    # Step 10: Return results for inspection
    return {
        "normalized_target": a,                  # Correct normalized input amplitudes
        "prepared_state": output,                # Actual state produced by your constructed circuit
        "difference_norm": LA.norm(output - a),  # How close they are (should be ~0)
        "theta": theta,                          # RY rotation angle used
        "U_full": U_full                         # Full 4x4 unitary matrix
    }

# -----------------------------
# 6. Example run
# -----------------------------
example_amps = [0.3+0.1j, 0.5, -0.4j, 0.1+0.2j] # Arbitrary complex amplitudes

# Prepare the state
res = prepare_two_qubit_state(example_amps)

# Print difference between the constructed and target states
print("Difference between prepared and target state:", res["difference_norm"])


Difference between prepared and target state: 1.2719202621569003e-16


In [4]:
# -------------------------------------------
# Test 1: Normalization Enforcement
# -------------------------------------------
def test_normalization_enforced():
    """
    Test whether the prepare_two_qubit_state() function:
    1. Automatically normalizes the input amplitudes.
    2. Produces an output state equal to the normalized target state.
    """

    # Define an unnormalized list of complex amplitudes
    # Note: these amplitudes do NOT satisfy |a0|^2 + |a1|^2 + |a2|^2 + |a3|^2 = 1
    amps = [1+1j, 2, -1j, 0.5]

    # Call the main function to prepare the state
    # The function internally normalizes the amplitudes
    result = prepare_two_qubit_state(amps)

    # Extract the prepared (output) state and the normalized target state
    out = result["prepared_state"]
    target = result["normalized_target"]

    # Assertion: the output state must be equal to the normalized target state
    # np.allclose(...) checks that two arrays are elementwise equal within a small tolerance
    # atol=1e-10 allows for tiny floating-point rounding errors
    assert np.allclose(out, target, atol=1e-10), "Prepared state != target"

    # If the assertion passes, print a success message
    print("Normalization test passed.")

# -------------------------------------------
# Test 2: Output Dimension Check
# -------------------------------------------
def test_output_dimension():
    """
    Test whether the output of prepare_two_qubit_state() has the correct dimension.
    A two-qubit system should always be represented by a 4-element vector.
    """

    # Define a simple amplitude list corresponding to |10⟩
    amps = [0,0,1,0]

    # Run the function
    result = prepare_two_qubit_state(amps)

    # Assert that the shape of the prepared state is (4,)
    # meaning it's a 1D vector with 4 entries → valid for 2 qubits
    assert result["prepared_state"].shape == (4,), "Output dimension incorrect."

    # Print confirmation if test passes
    print("Dimension test passed.")

# -------------------------------------------
# Run both tests
# -------------------------------------------

# This line executes the first test: normalization and correctness
test_normalization_enforced()

# This line executes the second test: output vector dimension
test_output_dimension()


Normalization test passed.
Dimension test passed.
